In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm_notebook

# 리뷰의 본문 링크 수집

In [ ]:
mallurl = 'https://instanttattoo.co.kr'

In [ ]:
# 카페24 리뷰 게시판에서 페이지번호를 변경하면서 게시물의 링크 수집

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
url = mallurl+'/board/product/list.html?board_no=4&product_category_depth1=&product_category_depth2=&product_category_depth3=&product_category_depth4=&page={}'

src_li = []
for page in tqdm_notebook(range(1, 4)):
    
    result = requests.get(url.format(page), headers=headers)
    soup = BeautifulSoup(result.text, 'lxml')
    tr_li = soup.select('tr.xans-record-')
    for tr in tr_li:
        alt_li = [node['alt'] for node in tr.select('img')]
        
        src = tr.select('.subject a:nth-of-type(2)')
        if len(src) > 0:
            src = src[0]['href']
    
        src_li.append([src, alt_li])  
    time.sleep(0.5)
    
src_df = pd.DataFrame(src_li, columns=['src', 'alt'])
src_df.to_csv('src_df.csv', index=False)

# 리뷰의 본문 내용 수집

In [ ]:
# 수집된 링크를 접속하여 본문을 수집
row_li = []
for idx in tqdm_notebook(src_df.index):
    item = src_df.loc[idx]['src']
    if type(item) == list:
        continue
        
    req = requests.get(mallurl + item, headers= headers)
    soup = BeautifulSoup(req.text)
    
    try:
        prdName = soup.select_one('.information a').text
    except:
        prdName = ''
    content = soup.select_one('.typeWrite div.detail')
    imgs = content.select('img')
    img_src_li = []
    for img in imgs:
        src = img['src']
        img_src_li.append(src)

    context_text = content.text

    etc = soup.select_one('.etcArea')

    row = [prdName, img_src_li, context_text, etc]
    row_li.append(row)
    
rdf = pd.DataFrame(row_li,columns=['prdname','img','text','etc'])
rdf.to_excel('rdf.xlsx', index=False)